In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from time import time
from tqdm import tqdm

In [17]:
device = torch.device("mps")

In [35]:
# read data

rating_df = pd.read_csv('rating.csv', parse_dates=['timestamp'])

In [36]:
rand_users = np.random.choice(rating_df['userId'].unique(),
                              size=int(len(rating_df['userId'].unique())*0.05),
                              replace=False)

In [37]:
rating_df = rating_df.loc[rating_df['userId'].isin(rand_users)]

print('Reduced dataframe: {} rows for {} different users'.format(len(rating_df), len(rand_users)))

Reduced dataframe: 1038616 rows for 6924 different users


In [38]:
# Get number of unique users and items
num_users = rating_df['userId'].nunique()
num_items = rating_df['movieId'].nunique()

# Mapping userId and movieId to indices
user2idx = {userId: idx for idx, userId in enumerate(rating_df['userId'].unique())}
item2idx = {movieId: idx for idx, movieId in enumerate(rating_df['movieId'].unique())}

# Apply the mapping to your dataframe
rating_df['userId'] = rating_df['userId'].map(user2idx)
rating_df['movieId'] = rating_df['movieId'].map(item2idx)

# Split the dataset into training and testing sets
train_data, test_data = train_test_split(rating_df, test_size=0.2, random_state=42)

In [39]:
class CFData(Dataset):
    def __init__(self, data):
        self.user_ids = torch.tensor(data['userId'].values, dtype=torch.long)
        self.item_ids = torch.tensor(data['movieId'].values, dtype=torch.long)
        self.ratings = torch.tensor(data['rating'].values, dtype=torch.float32)

    def __len__(self):
        return len(self.user_ids)

    def __getitem__(self, idx):
        return self.user_ids[idx], self.item_ids[idx], self.ratings[idx]

# Create DataLoader for training
train_dataset = CFData(train_data)
test_dataset = CFData(test_data)

train_loader = DataLoader(train_dataset, batch_size=4096, shuffle=True)

In [40]:
class MatrixFactorization(nn.Module):
    def __init__(self, num_users, num_items, latent_dim, regs=[0, 0]):
        super(MatrixFactorization, self).__init__()
        self.user_embedding = nn.Embedding(num_users, latent_dim)
        self.item_embedding = nn.Embedding(num_items, latent_dim)
        self.user_embedding.weight = nn.Parameter(self.user_embedding.weight * (1 - regs[0]))
        self.item_embedding.weight = nn.Parameter(self.item_embedding.weight * (1 - regs[1]))

    def forward(self, user, item):
        user_vecs = self.user_embedding(user)
        item_vecs = self.item_embedding(item)
        y_hat = torch.sum(user_vecs * item_vecs, dim=1)
        return y_hat

# Initialize the model
latent_dim = 15
model = MatrixFactorization(num_users, num_items, latent_dim).to(device)

In [51]:
def train_model(model, train_loader, device, epochs=100, learning_rate=0.001):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    hr_list = []
    loss_list = []
    best_hr, best_iter = -1, -1
    patience = 5
    patience_count = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        # Add tqdm progress bar
        with tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs}") as pbar:
            for user, item, rating in train_loader:
                # Move data to the MPS or CPU device
                user = user.to(device)
                item = item.to(device)
                rating = rating.to(device)
                
                optimizer.zero_grad()
                prediction = model(user, item)
                loss = criterion(prediction, rating)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

                # Update the progress bar
                pbar.set_postfix({'Loss': loss.item()})
                pbar.update(1)

        # Early stopping logic
        # hr = evaluate_model(model, test_data, device, topK=5)  
        # hr_list.append(hr)
        # loss_list.append(total_loss / len(train_loader))

        # if hr < np.max(hr_list):
        #     patience_count += 1
        # else:
        #     patience_count = 0
        #     best_hr, best_iter = hr, epoch

        # if patience_count == patience:
        #     print(f"Early stopping at epoch {epoch}")
        #     break

        print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader)}')

    print("End.")

In [48]:
def evaluate_model(model, test_data, device, topK=10):
    hits = []

    # Generate movie_ids_unique and interacted_items if not already done
    movie_ids_unique = rating_df['movieId'].unique()
    interacted_items = rating_df.groupby('userId')['movieId'].apply(list).to_dict()
    
    with tqdm(total=len(test_data), desc="Evaluating") as pbar:
        for index, row in test_data.iterrows():
            user = torch.tensor([row['userId']]).long().to(device)
            true_item = row['movieId']
            negative_items = set(movie_ids_unique) - set(interacted_items[user.item()])  # Adjust according to your data

            test_items = list(np.random.choice(list(negative_items), size=99)) + [true_item]
            user_tensor = torch.tensor([user.item()] * 100).long().to(device)
            item_tensor = torch.tensor(test_items).long().to(device)

            with torch.no_grad():
                predictions = model(user_tensor, item_tensor)
            
            top_k_items = item_tensor[torch.argsort(predictions, descending=True)[:topK]].tolist()

            if true_item in top_k_items:
                hits.append(1)
            else:
                hits.append(0)

            # Update progress bar
            pbar.update(1)

    return np.mean(hits)

In [52]:
# Train the model
train_model(model, train_loader, device, epochs=100, learning_rate=0.001)

# Evaluate the model 
hr = evaluate_model(model, test_data, device, topK=10)
print(f'Final Hit Ratio@5: {hr:.4f}')

Epoch 1/100: 100%|██████████████████| 203/203 [00:05<00:00, 39.03it/s, Loss=5.6]


Epoch 1/100, Loss: 6.359740449877208


Epoch 2/100: 100%|█████████████████| 203/203 [00:05<00:00, 39.64it/s, Loss=4.63]


Epoch 2/100, Loss: 5.061100882262432


Epoch 3/100: 100%|█████████████████| 203/203 [00:05<00:00, 38.84it/s, Loss=3.77]


...

Epoch 99/100, Loss: 0.5104795158496631


Epoch 100/100: 100%|████████████████| 203/203 [00:05<00:00, 37.27it/s, Loss=0.5]


Epoch 100/100, Loss: 0.5093180353711979
End.


Evaluating: 100%|██████████████████████| 207724/207724 [10:31<00:00, 329.04it/s]

Final Hit Ratio@5: 0.0542
